In [9]:
# =====================================================
# MODELO CON PESOS COMPARTIDOS
# =====================================================

import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report


In [10]:
#-------------------------------------------------
# Cargar MNIST
# -------------------------------------------------

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype("float32") / 255.
X_test = X_test.astype("float32") / 255.

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)


In [11]:
def crear_pares_balanceado(X, y, n_pairs=20000):
    pares = []
    etiquetas = []
    
    # Construyo índice por clase
    clases = {}
    for i in range(10):
        clases[i] = np.where(y == i)[0]

    mitad = n_pairs // 2

    # ---------------------------
    # Pares iguales (positivos)
    # ---------------------------
    for _ in range(mitad):
        c = np.random.randint(0, 10)
        i1, i2 = np.random.choice(clases[c], size=2, replace=False)
        pares.append([X[i1], X[i2]])
        etiquetas.append(1)

    # ---------------------------
    # Pares distintos (negativos)
    # ---------------------------
    for _ in range(mitad):
        c1, c2 = np.random.choice(range(10), size=2, replace=False)
        i1 = np.random.choice(clases[c1])
        i2 = np.random.choice(clases[c2])
        pares.append([X[i1], X[i2]])
        etiquetas.append(0)

    # Mezclar para evitar orden fijo
    pares = np.array(pares)
    etiquetas = np.array(etiquetas)

    idx = np.random.permutation(n_pairs)
    return pares[idx], etiquetas[idx]


In [12]:
# -------------------------------------------------
# Crear pares
# -------------------------------------------------

def crear_pares(X, y):
    pares = []
    etiquetas = []

    for _ in range(20000):  
        idx1 = np.random.randint(0, len(X))
        idx2 = np.random.randint(0, len(X))

        pares.append([X[idx1], X[idx2]])
        etiquetas.append(1 if y[idx1] == y[idx2] else 0)

    return np.array(pares), np.array(etiquetas)


In [13]:
train_pairs, train_labels = crear_pares_balanceado(X_train, y_train)
test_pairs, test_labels = crear_pares_balanceado(X_test, y_test)

A_train = train_pairs[:, 0]
B_train = train_pairs[:, 1]
A_test = test_pairs[:, 0]
B_test = test_pairs[:, 1]

print("A_train --> ", A_train.shape)
print("B_train --> ", B_train.shape)
print("A_test  --> ", B_test.shape)
print("B_test  --> ", B_test.shape)

print("Pares iguales en train :", np.sum(train_labels == 1))
print("Pares distintos en train:", np.sum(train_labels == 0))

print("Pares iguales en test  :", np.sum(test_labels == 1))
print("Pares distintos en test :", np.sum(test_labels == 0))

A_train -->  (20000, 784)
B_train -->  (20000, 784)
A_test  -->  (20000, 784)
B_test  -->  (20000, 784)
Pares iguales en train : 10000
Pares distintos en train: 10000
Pares iguales en test  : 10000
Pares distintos en test : 10000


In [14]:
# -------------------------------------------------
# Modelo siamés
# -------------------------------------------------

shared_dense = Dense(64, activation="tanh")

input_a = Input(shape=(784,))
input_b = Input(shape=(784,))

output_a = shared_dense(input_a)
output_b = shared_dense(input_b)

merged = Concatenate()([output_a, output_b])
final = Dense(1, activation="sigmoid")(merged)

model = Model([input_a, input_b], final)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", "AUC"]   # <---- MÉTRICA EXTRA
)

model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 64)           50240       ['input_5[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 concatenate_2 (Concatenate)    (None, 128)          0           ['dense_4[0][0]',          

In [15]:
# -------------------------------------------------
# Entrenamiento
# -------------------------------------------------

model.fit(
    [A_train, B_train],
    train_labels,
    validation_data=([A_test, B_test], test_labels),
    epochs=10,
    batch_size=32
)


Epoch 1/10
625/625 [==============================] - 4s 4ms/step - loss: 0.7088 - accuracy: 0.5005 - auc: 0.4983 - val_loss: 0.7089 - val_accuracy: 0.4969 - val_auc: 0.5013
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6942 - accuracy: 0.5165 - auc: 0.5241 - val_loss: 0.6960 - val_accuracy: 0.5037 - val_auc: 0.5031
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.6913 - accuracy: 0.5325 - auc: 0.5444 - val_loss: 0.6990 - val_accuracy: 0.5043 - val_auc: 0.5031
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6877 - accuracy: 0.5386 - auc: 0.5581 - val_loss: 0.7010 - val_accuracy: 0.5080 - val_auc: 0.5017
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6850 - accuracy: 0.5466 - auc: 0.5713 - val_loss: 0.7020 - val_accuracy: 0.5061 - val_auc: 0.5017
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6799 - accuracy: 0.5609 - auc: 0.5885 - val_loss: 0.70

In [16]:
# -------------------------------------------------
# Evaluación final con métricas adicionales
# -------------------------------------------------

# Predicciones para AUC y matriz de confusión
y_pred_proba = model.predict([A_test, B_test])
y_pred = (y_pred_proba > 0.5).astype(int)

auc = roc_auc_score(test_labels, y_pred_proba)
cm = confusion_matrix(test_labels, y_pred)
cr = classification_report(test_labels, y_pred)

print("\n===== MÉTRICAS FINALES =====")
print(f"AUC FINAL: {auc:.4f}")
print("\nMatriz de Confusión:")
print(cm)
print("\nReporte de Clasificación:")
print(cr)


625/625 [==============================] - 1s 2ms/step

===== MÉTRICAS FINALES =====
AUC FINAL: 0.5001

Matriz de Confusión:
[[5454 4546]
 [5453 4547]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.50      0.55      0.52     10000
           1       0.50      0.45      0.48     10000

    accuracy                           0.50     20000
   macro avg       0.50      0.50      0.50     20000
weighted avg       0.50      0.50      0.50     20000

